In [1]:
# from bs4 import BeautifulSoup
# import requests
# import re
# url = "https://multiply.org.in/en/"

ModuleNotFoundError: No module named 'bs4'

In [9]:
# headers=({'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'})

In [10]:
# webpage=requests.get(url,headers=headers)

In [11]:
# soup=BeautifulSoup(webpage.content,'html.parser')

In [2]:
# soup

In [13]:
# making a list of major sites

In [3]:
list_of_dict=[{'text':'Introduction to Basics of Money','url':'https://multiply.org.in/en/guide/basics-of-money/introduction-to-basics-of-money/'},
               {'text':'Time Value of Money','url':'https://multiply.org.in/en/guide/factors-that-influence-the-value-of-money/factors-that-influence-the-value-of-money/'},
               {'text':'What is Financial Planning?','url':'https://multiply.org.in/en/guide/financial-planning-and-budgeting/what-is-financial-planning/'}]

In [4]:
print(list_of_dict)

[{'text': 'Introduction to Basics of Money', 'url': 'https://multiply.org.in/en/guide/basics-of-money/introduction-to-basics-of-money/'}, {'text': 'Time Value of Money', 'url': 'https://multiply.org.in/en/guide/factors-that-influence-the-value-of-money/factors-that-influence-the-value-of-money/'}, {'text': 'What is Financial Planning?', 'url': 'https://multiply.org.in/en/guide/financial-planning-and-budgeting/what-is-financial-planning/'}]


In [5]:
import pandas as pd

In [6]:
df=pd.DataFrame(list_of_dict)

In [7]:
df.shape

(3, 2)

In [8]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings("ignore", message="Unverified HTTPS request")
warnings.filterwarnings("ignore", message="InsecureRequestWarning")

In [9]:
from opensearchpy import OpenSearch

CLUSTER_URL = 'https://localhost:9200'

def get_os_client(cluster_url = CLUSTER_URL,
                  username='admin',
                  password='admin'):
    '''
    Get OpenSearch client
    :param cluster_url: cluster URL like https://ml-te-netwo-1s12ba42br23v-ff1736fa7db98ff2.elb.us-west-2.amazonaws.com:443
    :return: OpenSearch client
    '''
    client = OpenSearch(
        hosts=[cluster_url],
        http_auth=(username, password),
        verify_certs=False
    )
    return client

client = get_os_client()

/Users/vpatwal/Desktop/trying_summarization/.venv/lib/python3.10/site-packages/opensearchpy/connection/http_urllib3.py:208: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [10]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "msmarco-distilbert-base-tas-b"


In [11]:
model = SentenceTransformer(MODEL_NAME)

In [12]:
EMBEDDING_DIM = model.encode(["Sample sentence"])[0].shape[0]
print(EMBEDDING_DIM)#will be used ahead

768


In [13]:
index_name = "new_one"

index_body = {
  "settings": {
    "index": {
      "knn": True,
      "knn.algo_param.ef_search": 100
    }
  },
  "mappings": { #how do we store, 
    "properties": {
        "embedding": {
          "type": "knn_vector", #we are going to put 
          "dimension": EMBEDDING_DIM,
          "method": {
            "name": "hnsw",
            "space_type": "l2",
            "engine": "faiss",
            "parameters": {
              "ef_construction": 128,
              "m": 24
            }
         }
     }
    }
}}


In [14]:
response = client.indices.delete(index = index_name)

In [15]:
response = client.indices.create(index=index_name, body=index_body)

In [16]:
df.head()
df.columns

Index(['text', 'url'], dtype='object')

In [17]:
import pandas as pd

for idno, row in df.iterrows():
    print(f"Id: {idno}, Start: {row['time']}, Overview: {row['text']},url:{row['video_url']}")
    # original_title = row['time']
    overview = row['text']
    id = row['time']
    st_time=row['time']
    url=row['video_url']
    # Sentence transformer model takes list of documents as input and returns list of embeddings.
    embedding = model.encode([overview])[0]
    # We are inserting a data point with 3 attribute, "id", "text" and "embedding" as knn_vector type.
    my_doc = {"id": str(idno), "plot": overview, "embedding": embedding,"time" : st_time,"url":url}
    res = client.index(
        index=index_name,
        body=my_doc,
        id = str(idno),
        refresh = True
        )

KeyError: 'time'